# 1. Initialize Network

In [1]:
from random import seed
from random import random
import numpy as np
 
# 네트워크 초기 설정 : 입력/은닉/출력층 받음
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network
 
seed(1)
network = initialize_network(2, 1, 2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


### 결과 해석
입력층: 입력 노드(2)와 바이어스(1)

은닉층1: 노드(1)과 바이어스(1)

출력층: 노드(2)

# 2. Forward Propagation

In [2]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]  # 순전파 진행 
    return activation
    # 한 노드에 대한 계산을 수행하는 함수
    # bias를 마지막 가중치로 두었음!

def sigmoid(activation):
    return 1.0 / (1.0 + np.e**(-activation)) # 시그모이드 구현

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)  
            neuron['output'] = sigmoid(activation) # 마지막 층에서의 결과값 사용!
            # 나온 계산 값을 그대로 쓰나요? : 네!
            
            new_inputs.append(neuron['output']) # new_input은 다음 히든층에 들어갈 값이죠?
        inputs = new_inputs
    return inputs

**여기까지는 순전파 학습과정이었습니다. 이 과정이 끝나면 가중치가 바뀌나요?  
답변을 답변의 근거 코딩 결과와 함께 보여주세요.**

: 순전파에서는 가중치는 변하지 않음을 확인할 수 있음

In [3]:
row = [1, 0, None]
forward_propagate(network, row)

[0.6629970129852887, 0.7253160725279748]

# 3. Backward Propagate Error

In [4]:
def sigmoid_derivative(output):
    return output * (1.0 - output) 
    # 시그모이드 미분, 다시 말해 출력값의 기울기를 계산!

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):  #j는 neuron['weights'][j]에서의 뉴런의 가중치의index
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                    # 각 뉴런에 대해 계산된 오류 신호
                    # j는 출력층의 neuron['weights'][j]의 노드 가중치의 index
                errors.append(error) 
        else: # 역전파이므로 else문부터 실행됨!
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output']) 
                # 역전파시 오차는 어떻게 설정했나요?
                
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output']) 
            # 시그모이드 함수를 사용한 역전파 
            # delta : 각 노드에 대해 계산된 오류 저장

backward_propagate_error() 함수는 출력층부터 시작해서 역방향으로 계산해 나감.따라서 출력층의 노드에 은닉층 노드가 쓰일 수 있게 델타값을 먼저 계산하게 됨!

In [5]:
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'output': 0.7105668883115941, 'delta': -0.002711797799238243}]
[{'weights': [0.2550690257394217, 0.49543508709194095], 'output': 0.6629970129852887, 'delta': -0.14813473120687762}, {'weights': [0.4494910647887381, 0.651592972722763], 'output': 0.7253160725279748, 'delta': 0.05472601157879688}]


# 4. Train Network

-back propagation으로 각 노드의 오류를 계산하면 가중치 업데이트가 가능함. (weights_update())

In [6]:
def weights_update(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1] # 가중치에서 마지막 항목은 바이어스니까!
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
                # 여기서 l_rate는 가중치를 얼마나 바꿔야하는지를 나타냄
            neuron['weights'][-1] += l_rate * neuron['delta'] # 퍼셉트론 학습 규칙
   #### weights_update 함수는 순전파/역전파를 이미 수행했다고 가정함!      

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row) # 순전파 
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))]) # 예측값의 오차 합
            backward_propagate_error(network, expected)
            weights_update(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [7]:
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]

In [8]:
n_inputs = 2
    # 뉴럴렛의 입력노드로 뭐가 들어가죠? 그럼 입력 노드의 개수는? : 2개의 입력 노드이므로 바이어스까지 총 3개의 노드가 들어감!
n_outputs = 2 
    # 뉴럴렛의 출력노드의 개수는 뭐라고 했죠? : 출력층의 노드 수(여기서는 2개!)
network = initialize_network(n_inputs, 2, n_outputs)
   
train_network(network, dataset, 0.5, 20, n_outputs) # 자유롭게 설정하고 최적을 찾아보세요.

# 학습된(최적화)된 네트워크가 초기 네트워크와 달리 어떻게 변하였는지 출력하시오. (layer별로,hint : for문))
for layer in network: # 네트워크의 크기는 2
    layer.append #layer가 리스트니까!
layer

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132


[{'weights': [2.515394649397849, -0.3391927502445981, -0.9671565426390278],
  'output': 0.23648794202357587,
  'delta': -0.04270059278364587},
 {'weights': [-2.5584149848484263, 1.0036422106209193, 0.42383086467582776],
  'output': 0.7790535202438367,
  'delta': 0.03803132596437354}]

### 결과 해석
각 training epoch마다 제곱오차의 합을 출력함.

또한 epoch가 커질수록 점점 오차가 작아지는 것을 확인할 수 있음!

# 5. Prediction

지금까지 계산한 값을 토대로 해서 출력층 각각의 노드에 속하는 확률을 출력값으로 사용하자!

여기서 predict() 함수는 출력층 중 확률이 가장 큰 클래스의 index를 print해 줍니당(max(output)을 쓰는 이유)

또한 여기서 class들은 음이 아닌 정수값으로 return 되었다고 가정합니다!

In [9]:
# 학습한 네트워크로 예측값을 뽑아보자.

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs)) # 순전파 결과에서 어떤것이 최종 아웃풋이 되나요?

In [10]:
# 네트워크가 잘 학습되었는지 확인해보자. 

for row in dataset:
    prediction = predict(network, row) # 앞서 최적(학습)시킨 네트워크로 잘 학습되었는지 평가 
    print('실제값=%d, 예측값=%d' % (row[-1], prediction))

실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=0, 예측값=0
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1
실제값=1, 예측값=1


정확도 100% !! 를 확인할 수 있습니다